In [3]:
import requests
import json
import pandas as pd
import re
import time
from tqdm import tqdm
import os
from termcolor import colored

In [4]:
overpass_url = "http://overpass-api.de/api/interpreter"
# output_catalogue = os.getcwd()
output_catalogue = 'C:\\Users\\79811\\Documents\\Работа\\OpenStreetMap'

In [5]:
# Функция, которая проверяет доступность сервера для запроса и ждет когда он будет доступен.
# В API есть ограничение на количество запросов их периодичность, поэтому чтобы не напороться на обшибку необходимо подождать доступности
def waiting_for_request():
    overpass_test_url = "http://overpass-api.de/api/status"
    available_test_request = None
    i = 0
    while available_test_request is None:
        available_test_request = requests.get(overpass_test_url).text
        available_test_request = re.search(r'(\d+)\sslots\savailable',available_test_request)
        if (available_test_request is None):
#             print('No free attempts for query. Waiting 10 seconds for next try.')
            time.sleep(10)
            i += 1
            
#     print(available_test_request.group(1) + ' slots are available')
    return True

Countries

In [128]:
def get_countries():
    waiting_for_request()
    overpass_query1 = """
            [out:json];relation["admin_level"="2"];out;
            """
    response1 = requests.get(overpass_url, 
                            params={'data': overpass_query1})
    data1 = response1.json()

    waiting_for_request()
    overpass_query2 = """
            [out:json];node[place=country];out;
            """
    response2 = requests.get(overpass_url, 
                            params={'data': overpass_query2})
    data2 = response2.json()
    
    arr1 = []
    for element in data1['elements']:
        if ('name:ru' in element['tags']) and ("ISO3166-1:alpha2" in element['tags']):
            if ('short_name:ru' in element['tags']):
                arr1.append({'Страна': element['tags']['name:ru'],
                             'Страна_short': element['tags']['short_name:ru'],
                             'Страна_en': element['tags']['name:en'],
                             'alpha2' : element['tags']["ISO3166-1:alpha2"]})
            else:
                arr1.append({'Страна': element['tags']['name:ru'],
                         'Страна_short': element['tags']['name:ru'],
                         'Страна_en': element['tags']['name:en'],
                         'alpha2' : element['tags']["ISO3166-1:alpha2"]})
    arr2 = []
    for element in data2['elements']:
        if ('name:ru' in element['tags']) and ("country_code_iso3166_1_alpha_2" in element['tags']):
            if ('short_name:ru' in element['tags']):
                arr2.append({'Страна': element['tags']['name:ru'],
                             'Страна_short': element['tags']['short_name:ru'],
                             'Страна_en': element['tags']['name:en'],
                             'alpha2' : element['tags']["country_code_iso3166_1_alpha_2"]})
            else:
                arr2.append({'Страна': element['tags']['name:ru'],
                             'Страна_short': element['tags']['name:ru'],
                             'Страна_en': element['tags']['name:en'],
                             'alpha2' : element['tags']["country_code_iso3166_1_alpha_2"]})    
    DF = pd.concat([pd.DataFrame(arr1),pd.DataFrame(arr2)])
    DF = DF.drop_duplicates('alpha2').reset_index(drop=True)
    
#     DF[['Страна', 'Страна_short', 'alpha2']].to_csv(output_catalogue+'\\Countries_vocabulary_ru.csv', index=False, encoding="windows-1251")
    DF.to_csv(output_catalogue+'\\Countries_vocabulary.csv', index=False, encoding='utf-8-sig')
    return DF

In [129]:
DF_countries = get_countries()
DF_countries.head(3)

,Страна,Страна_short,Страна_en,alpha2
0,Андорра,Андорра,Andorra,AD
1,Словакия,Словакия,Slovakia,SK
2,Австрия,Австрия,Austria,AT


federal districts, Russia

In [6]:
def get_federal_districts_1():
    waiting_for_request()
    overpass_query_FD = """
            [out:json];relation['addr:country'=RU][admin_level=3];out;
            """
    response_FD = requests.get(overpass_url, 
                            params={'data': overpass_query_FD})
    data_FD = response_FD.json()
    
    arr_FD = []
    for element in data_FD['elements']:
        for ID in [i['ref'] for i in element['members'] if i['type']=='relation']:
            if ('name:ru' in element['tags']) and ("addr:country" in element['tags']):
                arr_FD.append({ 'Страна' : element['tags']["addr:country"],
                                'Федеральный округ': element['tags']['name:ru'],
                               'Inner_region_ID' : ID})
    DF_FD = pd.DataFrame(arr_FD)
    DF_FD[['Страна', 'Федеральный округ']].drop_duplicates().to_csv(output_catalogue+'\\Federal_districts_vocabulary.csv', index=False, encoding="windows-1251")
    return DF_FD

In [7]:
Fed_district_DF = get_federal_districts_1()
Fed_district_DF

,Страна,Федеральный округ,Inner_region_ID
0,RU,Северо-Кавказский федеральный округ,109877
1,RU,Северо-Кавказский федеральный округ,108081
2,RU,Северо-Кавказский федеральный округ,110032
3,RU,Северо-Кавказский федеральный округ,109878
4,RU,Северо-Кавказский федеральный округ,109879
...,...,...,...
79,RU,Дальневосточный федеральный округ,151233
80,RU,Дальневосточный федеральный округ,147167
81,RU,Дальневосточный федеральный округ,147166
82,RU,Дальневосточный федеральный округ,145730


regions of Russia

In [276]:
def get_regions_1():
        regions_id = 'relation('+');relation('.join(Fed_district_DF.Inner_region_ID.astype(str).to_list())+');'
        waiting_for_request()
        overpass_query_Reg = """
                [out:json];
                (regions_id
                );
                out;
                """.replace('regions_id',regions_id)
        response_Reg = requests.get(overpass_url, 
                                params={'data': overpass_query_Reg})
        data_Reg = response_Reg.json()
        
        Regions_DF = pd.DataFrame([{'Inner_region_ID':i['id'],'Регион':i['tags']['name']} for i in data_Reg['elements']]).drop_duplicates()
        Regions_DF = pd.merge(Fed_district_DF, Regions_DF, how='left', left_on='Inner_region_ID', right_on='Inner_region_ID').drop(columns= ['Inner_region_ID'])
        
        return Regions_DF

In [284]:
Regions_DF = get_regions_1()
Regions_DF

,Страна,Федеральный округ,Регион
0,RU,Северо-Кавказский федеральный округ,Чечня
1,RU,Северо-Кавказский федеральный округ,Ставропольский край
2,RU,Северо-Кавказский федеральный округ,Северная Осетия — Алания
3,RU,Северо-Кавказский федеральный округ,Карачаево-Черкесия
4,RU,Северо-Кавказский федеральный округ,Кабардино-Балкария
...,...,...,...
79,RU,Дальневосточный федеральный округ,Камчатский край
80,RU,Дальневосточный федеральный округ,Еврейская автономная область
81,RU,Дальневосточный федеральный округ,Амурская область
82,RU,Дальневосточный федеральный округ,Забайкальский край


In [290]:
Regions_DF.append(pd.DataFrame([{'Страна': 'RU',
                                    'Федеральный округ': 'Южный федеральный округ',
                                    'Регион': 'Республика Крым'}]),ignore_index =True)

,Страна,Федеральный округ,Регион
0,RU,Северо-Кавказский федеральный округ,Чечня
1,RU,Северо-Кавказский федеральный округ,Ставропольский край
2,RU,Северо-Кавказский федеральный округ,Северная Осетия — Алания
3,RU,Северо-Кавказский федеральный округ,Карачаево-Черкесия
4,RU,Северо-Кавказский федеральный округ,Кабардино-Балкария
...,...,...,...
80,RU,Дальневосточный федеральный округ,Еврейская автономная область
81,RU,Дальневосточный федеральный округ,Амурская область
82,RU,Дальневосточный федеральный округ,Забайкальский край
83,RU,Дальневосточный федеральный округ,Республика Бурятия


In [283]:
Regions_DF.groupby(Regions_DF['Федеральный округ']).count().sort_values('Регион', ascending=False)

,Страна,Регион
Федеральный округ,,
Центральный федеральный округ,18,18
Приволжский федеральный округ,14,14
Дальневосточный федеральный округ,11,11
Северо-Западный федеральный округ,11,11
Сибирский федеральный округ,10,10
Северо-Кавказский федеральный округ,7,7
Южный федеральный округ,7,7
Уральский федеральный округ,6,6


Cities:

In [279]:
def get_cities():
    waiting_for_request()
    
#     Закомментированный текст запроса, чтобы добавить в список маленькие города, если вдруг сильно будет нужно.    
#     overpass_query_Cities = """
#             [out:json];relation['addr:country'=RU]["place"~"city|town"];out;
#             """
    overpass_query_Cities = """
            [out:json];relation['addr:country'=RU]["place"~"city"];out;
            """
    response_Cities = requests.get(overpass_url, 
                            params={'data': overpass_query_Cities})
    data_Cities = response_Cities.json()
    arr_Cities = []
    for element in data_Cities['elements']:
        if ('name:ru' in element['tags']) & ('addr:region' in element['tags']):
            if element['tags']['name:ru'] != 'イシリクリ':
                arr_Cities.append({'Регион' : element['tags']['addr:region'], 
                                    'Город': element['tags']['name:ru']})
            else:
                arr_Cities.append({'Регион' : element['tags']['addr:region'], 
                                    'Город': element['tags']['name']})
        elif ('name' in element['tags']) & ('addr:region' in element['tags']):
            arr_Cities.append({'Регион' : element['tags']['addr:region'], 
                                'Город': element['tags']['name']})
    
#     создаем технические столбцы для сравнений
    Cities_DF = pd.DataFrame(arr_Cities)
    Cities_DF['Var1'] =  Cities_DF['Регион']
    Cities_DF['Var2'] =  Cities_DF['Регион']
    Cities_DF['Регион_test'] = Cities_DF['Регион'].isin(list(Regions_DF['Регион']))
    
    not_found = list(Cities_DF[Cities_DF.Регион_test == False].Регион)
    
    var1 = []
    var2= []
    for el in not_found:
        if re.search(r'Республика\s(.+)',el) is None:
            var1.append('Республика '+el)
            var2.append(None)
        else:
            var1.append(None)
            var2.append(re.search(r'Республика\s(.+)',el).group(1))
    
    Cities_DF.Var1[Cities_DF.Регион_test == False]  = var1
    Cities_DF.Var2[Cities_DF.Регион_test == False]  = var2
    
    Cities_DF['Регион_test1'] = Cities_DF['Var1'].isin(list(Regions_DF['Регион']))
    Cities_DF['Регион_test2'] = Cities_DF['Var2'].isin(list(Regions_DF['Регион']))
    Cities_DF[Cities_DF.Регион_test == False]

    LF = lambda row: row['Регион'] if row['Регион_test'] == True else (row['Var1'] if row['Регион_test1'] == True 
                                                                                     else (row['Var2'] if row['Регион_test2'] == True else None))

    Cities_DF['Регион'] = Cities_DF.apply(LF,axis=1)
    Cities_DF['Регион_test'] = Cities_DF['Регион'].isin(list(Regions_DF['Регион']))
    if Cities_DF[Cities_DF.Регион_test == False].shape[0] == 0:
        print('All cities are mapped')
        Cities_DF = Cities_DF.drop(columns = ['Var1', 'Var2', 'Регион_test', 'Регион_test1', 'Регион_test2'])
        
    DF = pd.merge(Cities_DF, Regions_DF, how='left').drop_duplicates()[['Город','Регион','Федеральный округ']]
    
    DF.to_csv(output_catalogue+'\\Cities_vocabulary.csv', index=False, encoding="windows-1251")
    
    return DF

In [280]:
Cities_DF =  get_cities()

D:\anaconda3\envs\WorkEnv\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\anaconda3\envs\WorkEnv\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [281]:
Cities_DF

,Город,Регион,Федеральный округ
0,Электросталь,Московская область,Центральный федеральный округ
1,Долгопрудный,Московская область,Центральный федеральный округ
2,Химки,Московская область,Центральный федеральный округ
3,Королёв,Московская область,Центральный федеральный округ
4,Колпино,Санкт-Петербург,Северо-Западный федеральный округ
...,...,...,...
124,Магадан,Магаданская область,Дальневосточный федеральный округ
125,Новокуйбышевск,Самарская область,Приволжский федеральный округ
126,Белгород,Белгородская область,Центральный федеральный округ
127,Воркута,Республика Коми,Северо-Западный федеральный округ
